In [1]:
from __future__ import division
import csv
import pandas as pd
import numpy as np
from sklearn import model_selection
from tpot import TPOTClassifier
import tools

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

file = "endometrium.csv"

f = open(file)
csv_f = csv.reader(f)
features = next(csv_f)
dataset = pd.read_csv(file, names=features, usecols=range(1,6097), dtype=np.float64, skiprows=1, low_memory=False)
# INITIALIZING, CLEANING, AND STRATIFYING DATASET
dataset['outcome'] = pd.to_numeric(dataset['outcome'], errors = 'coerce')
dataset.dropna(axis=1, thresh=2, inplace=True)
dataset.dropna(how='all',thresh = 20,inplace=True)
#dataset = dataset[np.isfinite(dataset).all(1)]

#
array_OG = dataset.values
wh_inf = np.isinf(array_OG)
array_OG[wh_inf]=0
wh_nan = np.isnan(array_OG)
array_OG[wh_nan]=0
print(dataset)
print(array_OG.shape)
X = array_OG[:,:6092]
Y = array_OG[:,6092]
# Y_og = Y_og.astype('int32')
X = X.astype('float64')
Y = Y.astype('int32')

validation_size = 0.30
seed = 7
# X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X_og, Y_og, test_size=validation_size)
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)
print(X_train.shape)

           radVar1   radVar2       radVar3   radVar4   radVar5   radVar6  \
0      9798.333333   10056.0   4605.342618  0.470013  0.017688  0.111166   
1     15741.000000   16040.0   5913.624260  0.375683  0.019529  0.135505   
2      8414.333333    8576.0   3127.396302  0.371675  0.027144  0.261784   
3      3880.000000    4040.0   2114.539901  0.544985  0.022513  0.180081   
4     11004.333330   11288.0   4994.883375  0.453901  0.017587  0.109902   
5     17986.000000   18216.0   5194.529575  0.288810  0.027104  0.261025   
6      4250.000000    4424.0   2565.628846  0.603677  0.018451  0.120962   
7       220.666667     304.0    375.346874  1.700968  0.017120  0.104142   
8      1958.000000    2080.0   1274.102930  0.650717  0.024290  0.209635   
9     93042.333330   93344.0  13220.881980  0.142095  0.034531  0.423674   
10     4437.000000    4584.0   1952.664949  0.440087  0.029012  0.299053   
11     2104.666667    2224.0   1410.781996  0.670311  0.022409  0.178418   
12    23736.

In [2]:
print(Y_test)
#0 : 374 1 : 331
train = Y_train[Y_train==0]
print(len(train))

[0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 1
 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0 0 1 0
 0 0 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 1
 0 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 0 0
 1 1 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 0 1
 0 1 1 1 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 1 1 0 0]
257


In [3]:
from __future__ import division
pipeline_optimizer = TPOTClassifier(generations=5, population_size=5, cv=5, verbosity=2, scoring='roc_auc',config_dict='TPOT light')
pipeline_optimizer.fit(X_train, Y_train)
Y_pred = pipeline_optimizer.predict(X_test)
print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
print("AUC: " + repr(roc_auc_score(Y_test, Y_pred)))
print("Sensitivity: " + repr(recall_score(Y_test, Y_pred)))
tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
print("Specificity: " + repr(tn / (tn + fp)))

pipeline_optimizer.export('tpot_exported_pipeline.py')

SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdPcHRpbWl6YXRpb24gUHJvZ3Jlc3MnLCBtYXg9MzAsIHN0eWxlPVByb2dyZXNzU3R5bGUoZGVzY3JpcHTigKY=


Generation 1 - Current best internal CV score: 0.763336382016
Generation 2 - Current best internal CV score: 0.763336382016
Generation 3 - Current best internal CV score: 0.763336382016
Generation 4 - Current best internal CV score: 0.780932345319
Generation 5 - Current best internal CV score: 0.780932345319

Best pipeline: KNeighborsClassifier(MaxAbsScaler(input_matrix), n_neighbors=35, p=1, weights=uniform)
Accuracy: 0.7311320754716981
Average Precision Score: 0.6229559748427673
Kappa: 0.4664429530201343
Hamming Loss: 0.2688679245283019
AUC: 0.7376068376068377
Sensitivity: 0.8
Specificity: 0.6752136752136753


In [ ]:
import joblib
joblib.dump(pipeline_optimizer.fitted_pipeline_,'best.pkl')

In [6]:
model = joblib.load('best.pkl')
Y_pred = model.predict(X_train)
print("Accuracy: " + repr(accuracy_score(Y_train,Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
print("Specificity: " + repr(tn / (tn + fp)))

Accuracy: 0.7058823529411765
Average Precision Score: 0.6228462826885346
Kappa: 0.4156523587257731
Hamming Loss: 0.29411764705882354
AUC: 0.7095891314383698
Sensitivity: 0.7966101694915254
Specificity: 0.622568093385214


In [11]:

pipeline_optimizer = TPOTClassifier(generations=20, population_size=50, cv=2, verbosity=2, scoring='roc_auc',config_dict='TPOT light')
pipeline_optimizer.fit(X_train, Y_train)
Y_pred = pipeline_optimizer.predict(X_test)
print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
print("AUC: " + repr(roc_auc_score(Y_test, Y_pred)))
print("Sensitivity: " + repr(recall_score(Y_test, Y_pred)))
tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
print("Specificity: " + repr(tn / (tn + fp)))

pipeline_optimizer.export('tpot_exported_pipeline1.py')

Generation 1 - Current best internal CV score: 0.7105750487329434
Generation 2 - Current best internal CV score: 0.7105750487329434
Generation 3 - Current best internal CV score: 0.7105750487329434
Generation 4 - Current best internal CV score: 0.7224508921877344
Generation 5 - Current best internal CV score: 0.7224508921877344
Generation 6 - Current best internal CV score: 0.7250599790073474
Generation 7 - Current best internal CV score: 0.7250599790073474
Generation 8 - Current best internal CV score: 0.7278977357924726
Generation 9 - Current best internal CV score: 0.7278977357924726
Generation 10 - Current best internal CV score: 0.7278977357924726
Generation 11 - Current best internal CV score: 0.7420527815264657
Generation 12 - Current best internal CV score: 0.7434472934472934
Generation 13 - Current best internal CV score: 0.7434472934472934
Generation 14 - Current best internal CV score: 0.7434472934472934
Generation 15 - Current best internal CV score: 0.7434472934472934
Gene